In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("GCSExample")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")

# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

#### read data from google bucket
# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

#  Google Storage File Path
gsc_file_path = 'gs://group6_chicagocrime/chicago_crimes2.csv'  #  use your gcp bucket name. Also upload sales.csv first

# Create data frame
df = spark.read.format("csv").option("header", "true") \
            .load(gsc_file_path)

df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [2]:
# import requiered functions

from pyspark.sql.functions import  expr , col , count , when , isnan
from pyspark.sql.functions import avg,sum,min,max,row_number 

In [3]:
## clean data using filter function to remove Null values. 

df_clean = df.filter( col("Community Area").isNotNull())
df_clean.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in ['Community Area']] # check for absence of any Null value in Community area Column
   ).show()

+--------------+
|Community Area|
+--------------+
|             0|
+--------------+



In [4]:
#get the result for bellow question:
#1- percentage of arrested criminals in each community area 

from pyspark.sql.functions import udf

NumberCrimeInArea =df.groupBy("Community Area").count()
NumberCrimeInArea.orderBy(expr("count")).show()

# calculate sum of all crimes in all areas 
sumAr = NumberCrimeInArea.select(sum("count"))
# extract the integer to use in the function 
sumArrested = sumAr.first()['sum(count)']

# define the function to calculate the percentage of Arreste criminals in each Community Area
def percentage(value):
    return ((value/sumArrested) * 100)

# register the function
percentageudf = udf(percentage)

# use the function with SQL quries to have the column of percentage of arrested criminals in each Community Areas 
NumberCrimeInAreas = NumberCrimeInArea.select(col("Community Area"),col("count"), percentageudf(col("count")))
NumberCrimeInAreas.show()
print(sumArrested)

+--------------+-----+
|Community Area|count|
+--------------+-----+
|           9.0|  258|
|          47.0|  389|
|          12.0|  450|
|          55.0|  516|
|          18.0|  587|
|          74.0|  619|
|          36.0|  636|
|          37.0|  843|
|          13.0|  844|
|          64.0|  942|
|          72.0|  991|
|          57.0|  999|
|          62.0| 1037|
|          50.0| 1049|
|          59.0| 1083|
|          34.0| 1085|
|          11.0| 1157|
|          54.0| 1189|
|          10.0| 1279|
|          52.0| 1319|
+--------------+-----+
only showing top 20 rows

+--------------+-----+-------------------+
|Community Area|count|  percentage(count)|
+--------------+-----+-------------------+
|           1.0| 3592| 1.3585219644106579|
|          75.0| 2088|   0.78969762296477|
|          50.0| 1049|0.39673984985155347|
|          22.0| 4809| 1.8188007034662734|
|          65.0| 2056| 0.7775949774020915|
|          38.0| 3159|  1.194758041640665|
|          66.0| 6228|  2.355477392

In [5]:
# write the NumberCrimeInAreas table in the BigQuery 

# Use the Cloud Storage bucket for temporary BigQuery export data used by the connector.
bucket = "group6_chicagocrime"  #  bucket for the assignment
spark.conf.set('temporaryGcsBucket', bucket)

# Setup hadoop fs configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

# Saving the data to BigQuery
NumberCrimeInAreas.write.format('bigquery') \
  .option('table', 'datatengineering-group6.ChicagoCrime.NumberCrimeInAreas') \
  .mode("overwrite") \
  .save()


In [6]:
#2- finding the most occurred crime in each area

from pyspark.sql.functions import col, to_date
from pyspark.sql.window import Window
from pyspark.sql.functions import desc
from pyspark.sql.functions import count


windowSpec = Window\
  .partitionBy("Community Area", "Primary Type")\
  .orderBy(desc("Primary Type"))\
  .rowsBetween(Window.unboundedPreceding, Window.currentRow)

dfWithcount1 = df_clean.withColumn("count",count("Primary Type").over(windowSpec))

dfgrouped= dfWithcount1.groupBy("Community Area", "Primary Type").agg(max("count"))

dfgrouped.show(5)
#dfgrouped.show(10)
df_gr=dfgrouped.groupBy("Community Area").agg(max("max(count)"))

df_gr.show(10)

df2 = dfgrouped.withColumnRenamed("Community Area", "Community")
featur = (df2["Community"] == df_gr["Community Area"]) & (df2["max(count)"] == df_gr["max(max(count))"])
df2.join(df_gr,featur ).drop("Community Area","max(max(count))").show(100)
MaxCrimeInArea= df2.join(df_gr,featur ).drop("Community Area","max(max(count))")


+--------------+-------------------+----------+
|Community Area|       Primary Type|max(count)|
+--------------+-------------------+----------+
|           1.0|              ARSON|         4|
|           1.0|            ASSAULT|       238|
|           1.0|            BATTERY|       675|
|           1.0|           BURGLARY|       166|
|           1.0|CRIM SEXUAL ASSAULT|        27|
+--------------+-------------------+----------+
only showing top 5 rows

+--------------+---------------+
|Community Area|max(max(count))|
+--------------+---------------+
|           1.0|            855|
|          50.0|            233|
|          75.0|            491|
|          22.0|           1624|
|          65.0|            655|
|          38.0|            698|
|          66.0|           1262|
|          20.0|            329|
|          15.0|            810|
|          45.0|            274|
+--------------+---------------+
only showing top 10 rows

+---------+------------+----------+
|Community|Primary 

In [7]:

# Saving the data to BigQuery
MaxCrimeInArea.write.format('bigquery') \
  .option('table', 'datatengineering-group6.ChicagoCrime.MaxCrimeInArea') \
  .mode("overwrite") \
  .save()


In [8]:
##3-the area with the number of crimes more than average
 
from pyspark.sql.functions import sum, sum_distinct, mean
from pyspark.sql.functions import lit

df_1 = df_clean.groupby("Community Area").agg(count("Primary Type"))
df_3 = df_1.withColumnRenamed("count(Primary Type)", "CountCrime")

df_3

TotalCrime=df_3.select(
    sum("CountCrime").alias("TotalCrime"))
#TotalCrime.select(expr("mean(count('Primary Type'))").alias("mean_TotalCrime")).show()
average = df_3.select(mean("CountCrime"))
average.show()
averageint = average.first()['avg(CountCrime)']
print(averageint)



df_final = df_3.where(df_3.CountCrime >= averageint)
df_final.show()

+------------------+
|   avg(CountCrime)|
+------------------+
|3433.8311688311687|
+------------------+

3433.8311688311687
+--------------+----------+
|Community Area|CountCrime|
+--------------+----------+
|           1.0|      3592|
|          22.0|      4809|
|          66.0|      6228|
|          15.0|      3454|
|          67.0|      7548|
|          44.0|      6051|
|          69.0|      6784|
|          25.0|     17414|
|          26.0|      6038|
|          29.0|      8433|
|          71.0|      7858|
|          46.0|      5093|
|          19.0|      4846|
|          23.0|      8203|
|           6.0|      5590|
|          32.0|      7596|
|           7.0|      3764|
|          27.0|      5463|
|          68.0|      7258|
|          24.0|      7087|
+--------------+----------+
only showing top 20 rows



In [9]:
#### write data on BIGQUERY

# Saving the data to BigQuery
df_final.write.format('bigquery') \
  .option('table', 'datatengineering-group6.ChicagoCrime.AreaWithCrimeMoreAvg') \
  .mode("overwrite") \
  .save()

In [11]:
## 4- which crimes are more likely to have criminals arrested

from pyspark.sql.functions import col,avg,sum,min,max,row_number , desc

dfAREST=dfWithcount1.select("Primary Type", "count")

windowSpecAgg  = Window.partitionBy("Primary Type").orderBy(col("count").desc())

dfARESTs = dfAREST.withColumn("row",row_number().over(windowSpecAgg)) \
        .withColumn("max", max(col("count")).over(windowSpecAgg))\
        .where(col("row")== 1).orderBy(desc("max")).select("Primary Type","max")
dfARESTs.show()

+--------------------+----+
|        Primary Type| max|
+--------------------+----+
|               THEFT|4216|
|             BATTERY|3564|
|           NARCOTICS|3277|
|     CRIMINAL DAMAGE|1542|
|       OTHER OFFENSE|1251|
|  DECEPTIVE PRACTICE|1214|
|             ASSAULT|1096|
|             ROBBERY| 771|
|            BURGLARY| 745|
| MOTOR VEHICLE THEFT| 655|
|   CRIMINAL TRESPASS| 354|
|        PROSTITUTION| 324|
|   WEAPONS VIOLATION| 266|
|PUBLIC PEACE VIOL...| 170|
|OFFENSE INVOLVING...| 157|
|INTERFERENCE WITH...| 150|
| CRIM SEXUAL ASSAULT| 103|
|            HOMICIDE|  50|
|            GAMBLING|  46|
|         SEX OFFENSE|  43|
+--------------------+----+
only showing top 20 rows



In [12]:
#### write data on BIGQUERY

# Saving the data to BigQuery
dfARESTs.write.format('bigquery') \
  .option('table', 'datatengineering-group6.ChicagoCrime.ArestedCriminalsPerCrime') \
  .mode("overwrite") \
  .save()

In [13]:
spark.stop()